In [4]:
!pip install tensorflow numpy matplotlib scikit-learn



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import re
import time
import gc
import random
import numpy as np
# Force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# Disable XLA, as it's GPU-focused
os.environ['TF_XLA_FLAGS'] = ''

import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Ensure mixed precision uses float32 globally on CPU
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('float32')

print("Using devices:", tf.config.get_visible_devices('GPU'), tf.config.get_visible_devices('CPU'))

def clear_memory():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

# ——— CHECKPOINT UTIL ———
def get_latest_checkpoint(dir_path):
    files = [f for f in os.listdir(dir_path) if f.startswith("model_checkpoint_amla_cucoo_epoch_")]
    print("Checkpoints found:", files)
    if not files:
        return None, 0
    epochs = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in files if re.search(r'epoch_(\d+)', f)]
    idx = max(range(len(epochs)), key=lambda i: epochs[i])
    return os.path.join(dir_path, files[idx]), epochs[idx]

# ——— USER SETTINGS ———
selected_optimizer = 'adam'
train_dir      = './amla_images/train'
validation_dir = './amla_images/val'
test_dir       = './amla_images/test'
checkpoint_dir = './checkpoints_amla_cucoo'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint_amla_cucoo_epoch_{epoch:02d}.keras')

# ——— DATA GENERATORS ———
batch_size = 2
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
val_gen   = datagen.flow_from_directory(validation_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
test_gen  = datagen.flow_from_directory(test_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary', shuffle=False)

# ——— MODEL UTILITIES ———
def freeze_all(model):
    for layer in model.layers:
        layer.trainable = False

def unfreeze_layers(model, mode):
    if mode == 'all':
        for layer in model.layers:
            layer.trainable = True
    elif mode == 'last5':
        for layer in model.layers[-5:]:
            layer.trainable = True

def build_model():
    inp = Input(shape=(224,224,3))
    vgg = VGG19(weights='imagenet', include_top=False, input_tensor=inp)
    dn  = DenseNet121(weights='imagenet', include_top=False, input_tensor=inp)
    freeze_all(vgg)
    freeze_all(dn)

    x1 = GlobalAveragePooling2D()(vgg.output)
    x2 = GlobalAveragePooling2D()(dn.output)
    x  = Concatenate()([x1, x2])
    x  = Dense(256, activation='relu')(x)
    out= Dense(1, activation='sigmoid', dtype='float32')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# ——— EVALUATION FUNCTION FOR CSO ———
def evaluate_lr(lr_val):
    lr = float(lr_val[0]) if isinstance(lr_val, (list, np.ndarray)) else float(lr_val)
    model = build_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(train_gen, validation_data=val_gen, epochs=1, steps_per_epoch=2, validation_steps=1, verbose=0)
    val_acc = history.history['val_accuracy'][-1]
    clear_memory()
    return -val_acc

class CuckooSearch:
    def __init__(self, objective_function, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=10):
        self.f = objective_function
        self.n = n
        self.pa = pa
        self.alpha = alpha
        self.lb = np.array(lower_bound)
        self.ub = np.array(upper_bound)
        self.max_iter = max_iter
        self.d = len(lower_bound)

    def levy_flight(self):
        from scipy.special import gamma
        beta = 1.5
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
        u = np.random.randn(self.d) * sigma
        v = np.random.randn(self.d)
        step = u / np.abs(v)**(1 / beta)
        return step

    def search(self):
        nests = np.random.uniform(self.lb, self.ub, size=(self.n, self.d))
        fitness = np.array([self.f(nest) for nest in nests])
        best_idx = np.argmin(fitness)
        best_nest = nests[best_idx].copy()
        best_fitness = fitness[best_idx]

        for t in range(self.max_iter):
            for i in range(self.n):
                step = self.alpha * self.levy_flight()
                new_nest = nests[i] + step * (nests[i] - best_nest)
                new_nest = np.clip(new_nest, self.lb, self.ub)
                new_fit = self.f(new_nest)
                if new_fit < fitness[i]:
                    nests[i] = new_nest
                    fitness[i] = new_fit
                    if new_fit < best_fitness:
                        best_nest = new_nest
                        best_fitness = new_fit

            for i in range(self.n):
                if random.random() < self.pa:
                    nests[i] = np.random.uniform(self.lb, self.ub, self.d)
                    fitness[i] = self.f(nests[i])
                    if fitness[i] < best_fitness:
                        best_nest = nests[i]
                        best_fitness = fitness[i]

            print(f"[CS Iter {t+1}/{self.max_iter}] Best fitness = {-best_fitness:.4f}")

        return best_nest, best_fitness

def run_cuckoo_search():
    print("🔍 Running Cuckoo Search to find best learning rate...")
    cs = CuckooSearch(evaluate_lr, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=5)
    best_nest, best_score = cs.search()
    best_lr = float(best_nest[0])
    print(f"✅ Best LR found: {best_lr:.6f} with validation accuracy {-best_score:.4f}")
    return best_lr

# ——— OPTIMIZER FACTORY ———
def get_opt(name, lr):
    if name == 'adamw':
        return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=1e-5)
    elif name == 'nadam':
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        return tf.keras.optimizers.Adam(learning_rate=lr)

# ——— LOAD FROM CHECKPOINT IF AVAILABLE ———
latest_ckpt, last_epoch = get_latest_checkpoint(checkpoint_dir)
if latest_ckpt and os.path.exists(latest_ckpt):
    model = load_model(latest_ckpt, compile=False)
    print(f"Resuming from epoch {last_epoch}")
    # Recompile after loading
    model.compile(optimizer=get_opt(selected_optimizer, 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
else:
    model = build_model()
    last_epoch = 0
    print("Starting fresh training")

ckpt_cb = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1)

# ——— TRAINING PHASES ———
lr_phase_0 = run_cuckoo_search()

phases = [
    (lr_phase_0, 10, []),
    (1e-5, 5, ['last5']),
    (1e-6, 5, ['all']),
]

histories = []

for idx, (lr, epochs, flags) in enumerate(phases):
    if last_epoch >= sum(p[1] for p in phases[:idx+1]):
        continue

    if 'all' in flags:
        unfreeze_layers(model, 'all')
    elif 'last5' in flags:
        unfreeze_layers(model, 'last5')

    model.compile(optimizer=get_opt(selected_optimizer, lr), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_gen, validation_data=val_gen, epochs=last_epoch + epochs, initial_epoch=last_epoch,
                        callbacks=[ckpt_cb], steps_per_epoch=train_gen.samples // batch_size,
                        validation_steps=val_gen.samples // batch_size)
    histories.append(history)
    last_epoch += epochs

    if idx < len(phases) - 1:
        latest_ckpt, _ = get_latest_checkpoint(checkpoint_dir)
        clear_memory()
        model = load_model(latest_ckpt, compile=False)
        model.compile(optimizer=get_opt(selected_optimizer, lr), loss='binary_crossentropy', metrics=['accuracy'])
        print(f"Cleared session and reloaded model from {latest_ckpt}")

clear_memory()


Using devices: [] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Found 11124 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Found 2386 images belonging to 2 classes.
Checkpoints found: ['model_checkpoint_amla_cucoo_epoch_01.keras', 'model_checkpoint_amla_cucoo_epoch_02.keras', 'model_checkpoint_amla_cucoo_epoch_03.keras', 'model_checkpoint_amla_cucoo_epoch_04.keras', 'model_checkpoint_amla_cucoo_epoch_05.keras']
Resuming from epoch 5
🔍 Running Cuckoo Search to find best learning rate...


C:\Users\kavit\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



[CS Iter 1/5] Best fitness = 1.0000
[CS Iter 2/5] Best fitness = 1.0000
[CS Iter 3/5] Best fitness = 1.0000
[CS Iter 4/5] Best fitness = 1.0000
[CS Iter 5/5] Best fitness = 1.0000
✅ Best LR found: 0.000244 with validation accuracy 1.0000
Epoch 6/15
5562/5562 ━━━━━━━━━━━━━━━━━━━━ 0s 357ms/step - accuracy: 0.9990 - loss: 0.0034
Epoch 6: saving model to ./checkpoints_amla_cucoo\model_checkpoint_amla_cucoo_epoch_06.keras
5562/5562 ━━━━━━━━━━━━━━━━━━━━ 2414s 432ms/step - accuracy: 0.9990 - loss: 0.0034 - val_accuracy: 0.9946 - val_loss: 0.0215
Epoch 7/15
5562/5562 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.9990 - loss: 0.0020
Epoch 7: saving model to ./checkpoints_amla_cucoo\model_checkpoint_amla_cucoo_epoch_07.keras
5562/5562 ━━━━━━━━━━━━━━━━━━━━ 2392s 430ms/step - accuracy: 0.9990 - loss: 0.0020 - val_accuracy: 0.9987 - val_loss: 0.0049
Epoch 8/15
5562/5562 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step - accuracy: 0.9995 - loss: 0.0013
Epoch 8: saving model to ./checkpoints_amla_cucoo\model_c

In [ ]:
[
            ("Amla", "images_for_app/AMLA.avif"),
            ("Rose", "images_for_app/Rose.jpg"),
            ("Guava", "images_for_app/guava.jpg"),
            ("Hibiscus", "images_for_app/hibiscus.jpg"),
            ("Lantana", "images_for_app/lantana.jpg"),
        ]